In [1]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
sidoGuDong = pd.read_csv("./data/sidoGuDong.csv")
sidoGuDong = sidoGuDong.drop(['Unnamed: 0'], axis=1)

In [6]:
driver = wb.Chrome()
driver.get("https://map.google.com/")


In [11]:
search = driver.find_element(By.CSS_SELECTOR, "#searchboxinput")
search.send_keys("광주광역시 서구 쌍촌동")
search.send_keys(Keys.ENTER)


In [12]:
search.clear()

In [13]:
url = driver.current_url
url

'https://www.google.com/maps/place/%EA%B4%91%EC%A3%BC%EA%B4%91%EC%97%AD%EC%8B%9C+%EC%84%9C%EA%B5%AC+%EC%8C%8D%EC%B4%8C%EB%8F%99/data=!4m6!3m5!1s0x35718956cd3ca165:0xcb3103f7cae49af0!8m2!3d35.1528754!4d126.8625264!16s%2Fg%2F1tfmfqb8?entry=ttu'

![image.png](attachment:image.png)
- 대한민국 위도 = 33 - 43
-          경도 = 124 - 132

In [14]:
lat_ind = url.index("!3d")
lng_ind = url.index("!4d")

lat = url[lat_ind+3:lat_ind+13]
lng = url[lng_ind+3:lng_ind+14]
print(f"lat : {lat}  lng : {lng}")

lat : 35.1528754  lng : 126.8625264


In [ ]:
sidoGuDong

In [15]:
### 크롤링

lat_list = []
lng_list = []

driver = wb.Chrome()
driver.get("https://map.google.com/")
time.sleep(1)
search = driver.find_element(By.CSS_SELECTOR, "#searchboxinput")


# for i in range(5) :
for i in range(len(sidoGuDong['시도'])):
    search.send_keys(f"{sidoGuDong['시도'][i]} {sidoGuDong['지역이름'][i]} {sidoGuDong['법정동'][i]}")
    search.send_keys(Keys.ENTER)
    time.sleep(2)
    url = driver.current_url

    
    try :
        lat_ind = url.index("!3d")
        lng_ind = url.index("!4d")
        lat = url[lat_ind+3:lat_ind+13]
        lng = url[lng_ind+3:lng_ind+14]
    except :
        lat=""
        lng=""
        


#     print(f"lat : {lat}  lng : {lng}")
    lat_list.append(lat)
    lng_list.append(lng)
    
    search.clear()
    
driver.close()
    



In [144]:
len(lat_list)  ### 0~1025 까지 저장됐음.  

1026

In [145]:
sidoGuDong.iloc[1024]

시도                 경남
지역이름              진해구
법정동               용원동
avg      19780.663808
count            1749
Name: 1024, dtype: object

In [146]:
sidoGuDong.iloc[1025]

시도                 경남
지역이름              진해구
법정동                이동
avg      22187.402863
count             489
Name: 1025, dtype: object

In [141]:
## 에러 테스트
search.clear()
search.send_keys(f"경남 진해구 이동")
search.send_keys(Keys.ENTER)
time.sleep(2)
url = driver.current_url

try :
    lat_ind = url.index("!3d")
    lng_ind = url.index("!4d")
    lat = url[lat_ind+3:lat_ind+13]
    lng = url[lng_ind+3:lng_ind+14]
    print(f"{lat} {lng}")
except :
    lat=""
    lng=""


In [142]:
## 인덱스 1025에서 경남 진해구 이동을 검색하니 이동하는 경로가 검색돼었음.
## 법정동이 이동인 케이스 확인
sidoGuDong[sidoGuDong['법정동'] =="이동"]
## 336 경기 상록구 이동은 정상 검색되는것을 확인
## 1221 경북 남구 이동도 정상 검색됨
## 혹시 이동 말고도 다른 문제가 있을수도 있음. 뒤로가기 추가해서 다시 돌리기.

,시도,지역이름,법정동,avg,count
336,경기,상록구,이동,36831.806283,382
1025,경남,진해구,이동,22187.402863,489
1221,경북,남구,이동,23274.218750,384


In [169]:
### 크롤링 ver2

# lat_list = []
# lng_list = []

driver = wb.Chrome()
driver.get("https://map.google.com/")
time.sleep(3)


## 에러는 인덱스 1026에서 났음. 1026부터 다시 시작.
for i in range(1026, len(sidoGuDong['시도'])):
    search = driver.find_element(By.CSS_SELECTOR, "#searchboxinput")
    search.clear()
    search.send_keys(f"{sidoGuDong['시도'][i]} {sidoGuDong['지역이름'][i]} {sidoGuDong['법정동'][i]}")
    search.send_keys(Keys.ENTER)
    time.sleep(3)
    
    url = driver.current_url
    try :
        lat_ind = url.index("!3d")
        lng_ind = url.index("!4d")
        lat = url[lat_ind+3:lat_ind+13]
        lng = url[lng_ind+3:lng_ind+14]
    except :
        lat=""
        lng=""
        
        
    lat_list.append(lat)
    lng_list.append(lng)
    
    driver.back()
    time.sleep(1)
    


In [170]:
len(lat_list)

3053

In [193]:
sidoGuDong["latitude"] = lat_list
sidoGuDong["longitude"] = lng_list

In [194]:
# ""값 확인 (데이터를 못가져올 경우 빈칸)

sidoGuDong[(sidoGuDong['latitude']=='')] 

## 확인 결과, 구글에서 검색시 여러군데가 검색됨. 위치를 하나 선택해야 함. url 포맷이 달라짐.
# 9개는 하나씩 넣어줄수 있음.

,시도,지역이름,법정동,avg,count,latitude,longitude
139,경기,광주시,능평동,47398.214286,56,,
148,경기,광주시,양벌동,37720.044643,224,,
164,경기,광주시,추자동,43343.750000,32,,
1025,경남,진해구,이동,22187.402863,489,,
1334,경북,칠곡군,북삼읍 숭오리,7647.651934,724,,
2205,울산,울주군,삼남면 신화리,33612.173913,46,,
2917,충북,상당구,남문로1가동,9115.888889,45,,
2924,충북,상당구,북문로2가동,9173.046154,130,,
2925,충북,상당구,북문로3가동,25789.687831,189,,


In [195]:
# 데이터값 검색해서 수작업 변경
sidoGuDong.loc[139,'latitude'] = '37.3504626'
sidoGuDong.loc[139,'longitude'] = '127.1480481'

sidoGuDong.loc[148,'latitude'] = '37.3790304'
sidoGuDong.loc[148,'longitude'] = '127.2586482'

sidoGuDong.loc[164,'latitude'] = '37.3626865'
sidoGuDong.loc[164,'longitude'] = '127.2223893'

sidoGuDong.loc[1025,'latitude'] = '35.1507339'
sidoGuDong.loc[1025,'longitude'] = '128.6967315'

sidoGuDong.loc[1334,'latitude'] = '36.0738843'
sidoGuDong.loc[1334,'longitude'] = '128.3134343'

sidoGuDong.loc[2205,'latitude'] = '35.5442922'
sidoGuDong.loc[2205,'longitude'] = '129.1292172'

sidoGuDong.loc[2917,'latitude'] = '36.6317495'
sidoGuDong.loc[2917,'longitude'] = '127.4906148'

sidoGuDong.loc[2924,'latitude'] = '36.6365634'
sidoGuDong.loc[2924,'longitude'] = '127.4896277'

sidoGuDong.loc[2925,'latitude'] = '36.6437173'
sidoGuDong.loc[2925,'longitude'] = '127.4861328'



In [206]:
## 다 float으로 변경가능한지 확인

sidoGuDong['latitude'].str.contains("!").sum()
sidoGuDong['longitude'].str.contains('!').sum()

301

In [216]:
## ! 를 빈칸으로 대체
sidoGuDong['latitude'] = sidoGuDong['latitude'].str.replace('!',"")
sidoGuDong['longitude'] = sidoGuDong['longitude'].str.replace('!',"")

## d 빈칸으로
sidoGuDong['latitude'] = sidoGuDong['latitude'].str.replace('d',"")
sidoGuDong['longitude'] = sidoGuDong['longitude'].str.replace('d',"")

0

In [217]:
## 저장
dic = {"lat" : lat_list, "lng":lng_list}
loc = pd.DataFrame(dic)
loc.to_csv("./data/latlng.csv")
sidoGuDong.to_csv("./data/sidoGuDong_latlng.csv")

# 시도구동 -> 시도동으로 데이터 바뀜. 다시 크롤링


In [188]:
sidoDong = pd.read_csv("./data/sidoDong_pop.csv")
sidoDong = sidoDong.drop(['Unnamed: 0'], axis=1)

In [189]:
sidoDong

,시도,법정동,평균거래금액,거래수,인구수
0,강원,간성읍 상리,10671.783217,286,132.0
1,강원,간성읍 신안리,9696.842105,475,3925.0
2,강원,갈말읍 군탄리,7677.113122,221,1776.0
3,강원,갈말읍 신철원리,4780.392157,51,2390.0
4,강원,갈말읍 지포리,14852.482759,290,3409.0
...,...,...,...,...,...
3565,충북,풍동,3754.583333,120,619.0
3566,충북,하소동,13745.535567,1940,11177.0
3567,충북,호암동,12706.314103,2496,21787.0
3568,충북,화산동,15258.074534,322,5994.0


In [190]:
## 기존에 크롤링 된거에서 겹치는거는 넘기기 
sidoGuDongPast = pd.read_csv("./data/sidoGuDong_latlng.csv")
sidoGuDongPast = sidoGuDongPast.drop(['Unnamed: 0'], axis=1)

In [191]:
sidoGuDongPast

,시도,지역이름,법정동,avg,count,latitude,longitude
0,강원,강릉시,강동면 안인리,3249.090909,11,37.732929,128.976246
1,강원,강릉시,견소동,18091.992786,1109,37.772679,128.943061
2,강원,강릉시,교동,19930.917241,3770,37.767435,128.888073
3,강원,강릉시,금학동,17376.470588,17,37.753709,128.896388
4,강원,강릉시,남문동,16083.673469,98,37.749459,128.889471
...,...,...,...,...,...,...,...
3048,충북,흥덕구,오송읍 오송리,11446.067416,89,36.607217,127.332320
3049,충북,흥덕구,오송읍 정중리,9464.866667,60,36.624302,127.301493
3050,충북,흥덕구,옥산면 가락리,21298.261318,1745,36.670489,127.384051
3051,충북,흥덕구,옥산면 오산리,6120.660897,1271,36.663684,127.370450


In [192]:
sidoDong['latitude'] = np.nan
sidoDong['longitude'] = np.nan
sidoDong

,시도,법정동,평균거래금액,거래수,인구수,latitude,longitude
0,강원,간성읍 상리,10671.783217,286,132.0,NaN,NaN
1,강원,간성읍 신안리,9696.842105,475,3925.0,NaN,NaN
2,강원,갈말읍 군탄리,7677.113122,221,1776.0,NaN,NaN
3,강원,갈말읍 신철원리,4780.392157,51,2390.0,NaN,NaN
4,강원,갈말읍 지포리,14852.482759,290,3409.0,NaN,NaN
...,...,...,...,...,...,...,...
3565,충북,풍동,3754.583333,120,619.0,NaN,NaN
3566,충북,하소동,13745.535567,1940,11177.0,NaN,NaN
3567,충북,호암동,12706.314103,2496,21787.0,NaN,NaN
3568,충북,화산동,15258.074534,322,5994.0,NaN,NaN


In [204]:
## 기존 크롤링 값들 채우기
for i in range(len(sidoDong)) :
    sido = sidoDong['시도'][i]
    bub = sidoDong['법정동'][i]
    
    x = sidoGuDongPast.loc[(sidoGuDongPast['시도'] == sido) & (sidoGuDongPast['법정동'] == bub), ['latitude']].sum()
    y = sidoGuDongPast.loc[(sidoGuDongPast['시도'] == sido) & (sidoGuDongPast['법정동'] == bub), ['longitude']].sum()
    
    sidoDong.iloc[i]['latitude'] = x
    sidoDong.iloc[i]['longitude'] = y

C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\2692749722.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoDong.iloc[i]['latitude'] = x
C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\2692749722.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoDong.iloc[i]['longitude'] = y
C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\2692749722.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoDong.iloc[i]['latitude'] = x
C:

C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\2692749722.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoDong.iloc[i]['latitude'] = x
C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\2692749722.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoDong.iloc[i]['longitude'] = y
C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\2692749722.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoDong.iloc[i]['latitude'] = x
C:

C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\2692749722.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoDong.iloc[i]['latitude'] = x
C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\2692749722.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoDong.iloc[i]['longitude'] = y
C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\2692749722.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoDong.iloc[i]['latitude'] = x
C:

C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\2692749722.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoDong.iloc[i]['latitude'] = x
C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\2692749722.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoDong.iloc[i]['longitude'] = y
C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\2692749722.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoDong.iloc[i]['latitude'] = x
C:

C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\2692749722.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoDong.iloc[i]['latitude'] = x
C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\2692749722.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoDong.iloc[i]['longitude'] = y
C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\2692749722.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoDong.iloc[i]['latitude'] = x
C:

...

C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\2692749722.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoDong.iloc[i]['latitude'] = x
C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\2692749722.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoDong.iloc[i]['longitude'] = y
C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\2692749722.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoDong.iloc[i]['latitude'] = x
C:

C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\2692749722.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoDong.iloc[i]['latitude'] = x
C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\2692749722.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoDong.iloc[i]['longitude'] = y
C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\2692749722.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoDong.iloc[i]['latitude'] = x
C:

C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\2692749722.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoDong.iloc[i]['latitude'] = x
C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\2692749722.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoDong.iloc[i]['longitude'] = y
C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\2692749722.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoDong.iloc[i]['latitude'] = x
C:

C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\2692749722.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoDong.iloc[i]['latitude'] = x
C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\2692749722.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoDong.iloc[i]['longitude'] = y
C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\2692749722.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoDong.iloc[i]['latitude'] = x
C:

C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\2692749722.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoDong.iloc[i]['latitude'] = x
C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\2692749722.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoDong.iloc[i]['longitude'] = y
C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\2692749722.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoDong.iloc[i]['latitude'] = x
C:

In [207]:
sidoTemp = sidoDong[sidoDong['latitude'] == 0.0]
sidoTemp


,시도,법정동,평균거래금액,거래수,인구수,latitude,longitude
0,강원,간성읍 상리,10671.783217,286,132.0,0.0,0.0
1,강원,간성읍 신안리,9696.842105,475,3925.0,0.0,0.0
2,강원,갈말읍 군탄리,7677.113122,221,1776.0,0.0,0.0
3,강원,갈말읍 신철원리,4780.392157,51,2390.0,0.0,0.0
4,강원,갈말읍 지포리,14852.482759,290,3409.0,0.0,0.0
...,...,...,...,...,...,...,...
3556,충북,진천읍 성석리,10309.853261,552,5343.0,0.0,0.0
3557,충북,진천읍 신정리,9599.123045,959,59.0,0.0,0.0
3558,충북,진천읍 읍내리,7747.154472,123,3568.0,0.0,0.0
3559,충북,진천읍 장관리,10733.447038,557,4588.0,0.0,0.0


In [ ]:
######## 추가 크롤링

In [209]:

driver = wb.Chrome()
driver.get("https://map.google.com/")
time.sleep(2)

### 로직 업데이트.


for i in range(len(sidoTemp)):
    search = driver.find_element(By.CSS_SELECTOR, "#searchboxinput")
    search.clear()
    search.send_keys(f"{sidoTemp.iloc[i]['시도']} {sidoTemp.iloc[i]['법정동']}")
    search.send_keys(Keys.ENTER)
    time.sleep(3)
    
    url = driver.current_url
    try :
        lat_ind = url.index("!3d")
        lng_ind = url.index("!4d")
        end_ind = url.index("!16s")
        
#         print(f"{lat_ind} {lng_ind} {end_ind}")
        
        lat = np.float64(float(url[lat_ind+3:lng_ind]))
        lng = np.float64(float(url[lng_ind+3:end_ind]))
    except :
        lat = 0.0
        lng = 0.0
        
#     print(f"{lat} {lng}")

    sidoTemp['latitude'].iloc[i] = lat
    sidoTemp['longitude'].iloc[i] = lng

    
    
    driver.back()
    time.sleep(1)
    
driver.close()

C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\3131258013.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoTemp['latitude'].iloc[i] = lat
C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\3131258013.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoTemp['longitude'].iloc[i] = lng
C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\3131258013.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoTemp['latitude'].iloc[i] 

C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\3131258013.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoTemp['latitude'].iloc[i] = lat
C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\3131258013.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoTemp['longitude'].iloc[i] = lng
C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\3131258013.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoTemp['latitude'].iloc[i] 

C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\3131258013.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoTemp['latitude'].iloc[i] = lat
C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\3131258013.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoTemp['longitude'].iloc[i] = lng
C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\3131258013.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoTemp['latitude'].iloc[i] 

C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\3131258013.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoTemp['latitude'].iloc[i] = lat
C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\3131258013.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoTemp['longitude'].iloc[i] = lng
C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\3131258013.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoTemp['latitude'].iloc[i] 

C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\3131258013.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoTemp['latitude'].iloc[i] = lat
C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\3131258013.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoTemp['longitude'].iloc[i] = lng
C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\3131258013.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoTemp['latitude'].iloc[i] 

...

C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\3131258013.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoTemp['latitude'].iloc[i] = lat
C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\3131258013.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoTemp['longitude'].iloc[i] = lng
C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\3131258013.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoTemp['latitude'].iloc[i] 

C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\3131258013.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoTemp['latitude'].iloc[i] = lat
C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\3131258013.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoTemp['longitude'].iloc[i] = lng
C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\3131258013.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoTemp['latitude'].iloc[i] 

C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\3131258013.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoTemp['latitude'].iloc[i] = lat
C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\3131258013.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoTemp['longitude'].iloc[i] = lng
C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\3131258013.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoTemp['latitude'].iloc[i] 

C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\3131258013.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoTemp['latitude'].iloc[i] = lat
C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\3131258013.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoTemp['longitude'].iloc[i] = lng
C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\3131258013.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoTemp['latitude'].iloc[i] 

C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\3131258013.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoTemp['latitude'].iloc[i] = lat
C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\3131258013.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoTemp['longitude'].iloc[i] = lng
C:\Users\skyfa\AppData\Local\Temp\ipykernel_2936\3131258013.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidoTemp['latitude'].iloc[i] 

In [98]:
## 코드 로직 테스트
url = "https://www.google.com/maps/place/%EA%B0%95%EC%9B%90%EB%8F%84+%EA%B3%A0%EC%84%B1%EA%B5%B0+%EA%B0%84%EC%84%B1%EC%9D%8D+%EC%83%81%EB%A6%AC/data=!4m6!3m5!1s0x5fd8f2843b9e3169:0xb76f5252631a5ed4!8m2!3d38.3825376!4d128.4630941!16s%2Fg%2F1tq6jxxw?entry=ttu"

lat_ind = url.index("!3d")
lng_ind = url.index("!4d")
end_ind = url.index("!16s")

print(f"{lat_ind} {lng_ind} {end_ind}")
print(url[lat_ind+3:lng_ind])
print(url[lng_ind+3:end_ind])

lat = np.float64(float(url[lat_ind+3:lng_ind]))
lng = np.float64(float(url[lng_ind+3:end_ind]))


194 207 221
38.3825376
128.4630941
38.3825376 128.4630941


In [211]:
backup = sidoTemp.copy()
sidoTemp

,시도,법정동,평균거래금액,거래수,인구수,latitude,longitude
0,강원,간성읍 상리,10671.783217,286,132.0,38.382538,128.463094
1,강원,간성읍 신안리,9696.842105,475,3925.0,38.379239,128.475644
2,강원,갈말읍 군탄리,7677.113122,221,1776.0,38.159236,127.297865
3,강원,갈말읍 신철원리,4780.392157,51,2390.0,38.128521,127.341546
4,강원,갈말읍 지포리,14852.482759,290,3409.0,38.138726,127.298195
...,...,...,...,...,...,...,...
3556,충북,진천읍 성석리,10309.853261,552,5343.0,36.864445,127.453135
3557,충북,진천읍 신정리,9599.123045,959,59.0,36.848337,127.455568
3558,충북,진천읍 읍내리,7747.154472,123,3568.0,36.857496,127.443458
3559,충북,진천읍 장관리,10733.447038,557,4588.0,36.871707,127.426609


In [215]:
## 데이터 전처리
# sidoTemp.query('latitude == 0.0')
# sidoTemp.query('longitude == 0.0')
sidoTemp.isna().sum()

시도           0
법정동          0
평균거래금액       0
거래수          0
인구수          0
latitude     0
longitude    0
dtype: int64

In [216]:
## 새로 크롤링한 데이터를 기존 데이터로 통합
for i in range(len(sidoTemp)) :
    sido = sidoTemp.iloc[i]['시도']
    bub = sidoTemp.iloc[i]['법정동']
    


    sidoDong.loc[((sidoDong['시도'] == sido) & (sidoDong['법정동'] == bub)), 'latitude'] = sidoTemp.iloc[i]['latitude']
    sidoDong.loc[((sidoDong['시도'] == sido) & (sidoDong['법정동'] == bub)), 'longitude'] = sidoTemp.iloc[i]['longitude']

#     print(f"{sidoTemp.iloc[i]['latitude']}  {sidoTemp.iloc[i]['longitude']}")
    
               

In [217]:
sidoDong

,시도,법정동,평균거래금액,거래수,인구수,latitude,longitude
0,강원,간성읍 상리,10671.783217,286,132.0,38.382538,128.463094
1,강원,간성읍 신안리,9696.842105,475,3925.0,38.379239,128.475644
2,강원,갈말읍 군탄리,7677.113122,221,1776.0,38.159236,127.297865
3,강원,갈말읍 신철원리,4780.392157,51,2390.0,38.128521,127.341546
4,강원,갈말읍 지포리,14852.482759,290,3409.0,38.138726,127.298195
...,...,...,...,...,...,...,...
3565,충북,풍동,3754.583333,120,619.0,36.929005,127.910564
3566,충북,하소동,13745.535567,1940,11177.0,37.140151,128.194711
3567,충북,호암동,12706.314103,2496,21787.0,36.949559,127.941793
3568,충북,화산동,15258.074534,322,5994.0,37.111973,128.203791


In [220]:
# 데이터 체크
# sidoDong[sidoDong['latitude'] == 0.0]
# sidoDong[sidoDong['longitude'] == 0.0]
sidoDong.isna().sum()

시도           0
법정동          0
평균거래금액       0
거래수          0
인구수          3
latitude     0
longitude    0
dtype: int64

In [221]:
sidoDong[sidoDong['인구수'].isna()]

,시도,법정동,평균거래금액,거래수,인구수,latitude,longitude
3401,충북,남문로1가동,9115.888889,45,NaN,36.631749,127.490615
3457,충북,북문로2가동,9173.046154,130,NaN,36.636563,127.489628
3458,충북,북문로3가동,25789.687831,189,NaN,36.643717,127.486133


In [223]:
### dongInGu 에서 검색
## 남문로1가 인구수 : 667
# 북문로2가동 인구수 : 737
#   북문로3가동 인구수 :   2447
# float으로

sidoDong.loc[((sidoDong['시도'] == '충북') & (sidoDong['법정동'] == '남문로1가동')), '인구수'] = 667.0
sidoDong.loc[((sidoDong['시도'] == '충북') & (sidoDong['법정동'] == '북문로2가동')), '인구수'] = 737.0
sidoDong.loc[((sidoDong['시도'] == '충북') & (sidoDong['법정동'] == '북문로3가동')), '인구수'] = 2447.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3570 entries, 0 to 3569
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   시도         3570 non-null   object 
 1   법정동        3570 non-null   object 
 2   평균거래금액     3570 non-null   float64
 3   거래수        3570 non-null   int64  
 4   인구수        3567 non-null   float64
 5   latitude   3570 non-null   float64
 6   longitude  3570 non-null   float64
dtypes: float64(4), int64(1), object(2)
memory usage: 195.4+ KB


In [224]:
sidoDong.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3570 entries, 0 to 3569
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   시도         3570 non-null   object 
 1   법정동        3570 non-null   object 
 2   평균거래금액     3570 non-null   float64
 3   거래수        3570 non-null   int64  
 4   인구수        3570 non-null   float64
 5   latitude   3570 non-null   float64
 6   longitude  3570 non-null   float64
dtypes: float64(4), int64(1), object(2)
memory usage: 195.4+ KB


## 최종 데이터 저장

In [225]:
sidoDong.to_csv("./data/sidoDong_pop_latlng.csv")

In [4]:
sidoDong = pd.read_csv("./data/sidoDong_pop_latlng.csv")

In [6]:
sidoDongUpdated = pd.read_csv("./data/sidoDong_pop.csv")

In [8]:
sidoDongUpdated.drop(columns=['Unnamed: 0',], inplace=True)
sidoDongUpdated

,시도,법정동,평균거래금액,거래수,평균전용면적,인구수
0,강원,간성읍 상리,10671.783217,286,62.814196,132.0
1,강원,간성읍 신안리,9696.842105,475,56.012926,3925.0
2,강원,갈말읍 군탄리,7677.113122,221,55.243620,1776.0
3,강원,갈말읍 신철원리,4780.392157,51,63.961176,2390.0
4,강원,갈말읍 지포리,14852.482759,290,88.535064,3409.0
...,...,...,...,...,...,...
3565,충북,풍동,3754.583333,120,38.130583,619.0
3566,충북,하소동,13745.535567,1940,70.713010,11177.0
3567,충북,호암동,12706.314103,2496,66.323622,21787.0
3568,충북,화산동,15258.074534,322,74.001727,5994.0


In [9]:
sidoDong.drop(columns=['Unnamed: 0'], inplace=True)

In [12]:
updatedSidoDong = sidoDongUpdated.merge(sidoDong, how='inner')
updatedSidoDong

,시도,법정동,평균거래금액,거래수,평균전용면적,인구수,latitude,longitude
0,강원,간성읍 상리,10671.783217,286,62.814196,132.0,38.382538,128.463094
1,강원,간성읍 신안리,9696.842105,475,56.012926,3925.0,38.379239,128.475644
2,강원,갈말읍 군탄리,7677.113122,221,55.243620,1776.0,38.159236,127.297865
3,강원,갈말읍 신철원리,4780.392157,51,63.961176,2390.0,38.128521,127.341546
4,강원,갈말읍 지포리,14852.482759,290,88.535064,3409.0,38.138726,127.298195
...,...,...,...,...,...,...,...,...
3517,충북,풍동,3754.583333,120,38.130583,619.0,36.929005,127.910564
3518,충북,하소동,13745.535567,1940,70.713010,11177.0,37.140151,128.194711
3519,충북,호암동,12706.314103,2496,66.323622,21787.0,36.949559,127.941793
3520,충북,화산동,15258.074534,322,74.001727,5994.0,37.111973,128.203791


In [13]:
updatedSidoDong.to_csv("./data/sidoDong_pop_latlng.csv")